#Generate Training and Validation Dataset (Only need to run if you don't have it)

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd

# Authenticate
drive = None
def authenticate():
  global drive
  
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

#Download files
def downloadFiles(fileIds):
  authenticate()
  
  for fileId in fileIds:    
    
    downloaded = drive.CreateFile({"id": fileId[1]})
    downloaded.GetContentFile(fileId[0])

In [ ]:
#original data
try:
  _ = open("DL_dataset.csv", "r")
except:
    downloadFiles([["DL_dataset.csv", "1-8WAgkLmCpk6_q1-hjgSapyrIKZi8xsW"]])



KeyboardInterrupt: ignored

In [ ]:
df_data = pd.read_csv('DL_dataset.csv')
df_data["label"].value_counts()

In [ ]:
label_transform = {"label": {"title":0, "section":1, "table":2, "text":3}}
df_data = df_data.replace(label_transform)
df_data["label"].value_counts()

In [ ]:
train_df, test_df = np.split(df_data.sample(frac=1, random_state=42), [int(.7*len(df_data))])

new_df = train_df[['text', 'label','visual_feature']]
new_df_test = test_df[['text', 'label','visual_feature']]
print(len(new_df))
print(len(new_df_test))

In [ ]:
new_df.to_csv("/content/train_data2.csv",index=False)
new_df_test.to_csv("/content/test_data2.csv", index=False)

# Classification With Visual Feature!

## Environment Setup
Import key libraries and working envorinments. 

In [ ]:
!pip install transformers==4.2.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.8 MB 30.7 MB/s 
     |████████████████████████████████| 2.9 MB 64.0 MB/s 
     |████████████████████████████████| 880 kB 64.9 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=a0f93649d25b35451f9c69fc8ced7e83e09d93388c11d39b16f85f2d6bdd707b
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [ ]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, RobertaModel, AutoModel
#from transformers.configuration_bert import BertConfig
import logging
logging.basicConfig(level=logging.ERROR)

In [ ]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [ ]:
#check cuda memory
import gc
gc.collect()
with torch.no_grad():
    torch.cuda.empty_cache()
!nvidia-smi

Fri May 27 03:23:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd

# Authenticate
drive = None
def authenticate():
  global drive
  
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

#Download files
def downloadFiles(fileIds):
  authenticate()
  
  for fileId in fileIds:    
    
    downloaded = drive.CreateFile({"id": fileId[1]})
    downloaded.GetContentFile(fileId[0])

##Get Training and Validation Dataset

In [ ]:
try:
  _ = open("test_data.csv", "r")
except:
  downloadFiles([["test_data.csv", "1--GgJAPbZUB1HN00TIzoAFQMfGpxq1GX"]])

try:
  _ = open("train_data.csv", "r")
except:
  downloadFiles([["train_data.csv", "1-6d07mw_MfdFMG1UQE62L1xCf2_bu_5D"]])

new_df = pd.read_csv('train_data.csv')

new_df_test = pd.read_csv('test_data.csv')

In [ ]:
#clean visual features
for i in range(len(new_df)):
  new_df['visual_feature'][i] = list(map(float,new_df['visual_feature'][i].replace('[','').replace(']','').split(', ')))

for i in range(len(new_df_test)):
  new_df_test['visual_feature'][i] = list(map(float,new_df_test['visual_feature'][i].replace('[','').replace(']','').split(', ')))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Data Preprocessing

In [ ]:
from transformers import AutoTokenizer
import torch


tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# Defining some key variables that will be used later on in the training
MAX_LEN = 100
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = TRAIN_BATCH_SIZE*2
# EPOCHS = 1
LEARNING_RATE = 2e-05
# Change the pre-trained bert model
#tokenizer = BertTokenizer.from_pretrained('roberta-base') #Cased 

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.label
        self.max_len = max_len
        self.visual = self.data.visual_feature

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float),
            'visual': torch.tensor(self.visual[index],dtype=torch.float),
        }

In [ ]:
train_size = 1
train_data=new_df.sample(frac=train_size,random_state=200)
train_data = train_data.reset_index(drop=True)

print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(new_df_test.shape))

training_set = SentimentData(train_data, tokenizer, MAX_LEN)
test_set = SentimentData(new_df_test,tokenizer,MAX_LEN)

TRAIN Dataset: (60202, 3)
TEST Dataset: (25801, 3)


In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
vali_loader = DataLoader(test_set, **test_params)

## Define the proposed classifiers

In [ ]:
class BertClass(torch.nn.Module):
    def __init__(self):
        super(BertClass, self).__init__()
        #BERT/BERT Large/Roberta/LayoutLM/Distill BERT/ALBERT

        self.l1 = AutoModel.from_pretrained('bert-base-uncased')# BERT large
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.visual_projection = torch.nn.Linear(2048,768)
        self.linear = torch.nn.Linear(768*2,768)
        self.classifier = torch.nn.Linear(768, 4)


    def forward(self,input_ids,attention_mask,token_type_ids,visual):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0] 

        visual = self.visual_projection(visual)
        visual = torch.nn.Tanh()(visual)
        visual = self.dropout(visual)


        pooler = torch.cat((pooler,visual),1)
        pooler = self.linear(pooler)
        pooler = torch.nn.Tanh()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output

In [ ]:
model = BertClass()
model.to(device)

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertClass(
  (l1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    

## Training 

In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=1e-05) # change learning rate

def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model

def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    output = []
    model.train()
    count = 0
    for _,data in tqdm(enumerate(training_loader, 0)):
        count += 1
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)
        visual = data['visual'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids,visual)

        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()
        if isinstance(BATCH_NUM,int) and count > BATCH_NUM:
          break

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")
    torch.save(model.state_dict(), '/content/model_visual.pt')

    return

## Testing

In [ ]:
class SentimentData_test(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.max_len = max_len
        self.visual = dataframe.visual_feature

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'visual': torch.tensor(self.visual[index], dtype=torch.float),
        }

### load the test datasets

In [ ]:
new_df_true_test = new_df_test[['text','visual_feature']]

In [ ]:
test = SentimentData_test(new_df_true_test,tokenizer, MAX_LEN)
testing_loader = DataLoader(test, **test_params)

In [ ]:
def test_label_generator(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    output_list = []
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)
            visual = data['visual'].to(device, dtype=torch.float)
            outputs = model(ids, mask, token_type_ids,visual)
            # outputs = model(ids, mask, token_type_ids,visual).squeeze()
            
            big_val, big_idx = torch.max(outputs.data, dim=1)
            output_list = output_list + list(big_idx)

            nb_tr_steps += 1
            
    return output_list


## Train & Test

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

EPOCHS = 15
BATCH_NUM = 'all'
# BATCH_NUM = int(len(training_loader)/2)

for epoch in range(11,EPOCHS+1):
  output = train(epoch)
  test_output = test_label_generator(model, testing_loader)
  # Get the predicted category id for selected test dataset.
  q = []
  for p in test_output:
    q.append(p.cpu().numpy().tolist())
  p = new_df_test['label'].tolist()
  report = classification_report(p,q, digits=4)
  #matrix = confusion_matrix(p,q)
  print(report)


0it [00:00, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2143: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
1489it [07:06,  3.49it/s]

In [31]:
import shutil
shutil.copy("/content/train_data.csv", "/content/drive/MyDrive/ste/train_data2.csv")

'/content/drive/MyDrive/ste/train_data2.csv'

In [ ]:
# output = test_label_generator(model, testing_loader)

In [ ]:
# Get the predicted category id for selected test dataset.
# q = []
# for p in output:
#   q.append(p.cpu().numpy().tolist())
# print(q)

In [ ]:
# p = new_df_test['label'].tolist()

In [ ]:
# from sklearn.metrics import classification_report, confusion_matrix
# report = classification_report(p,q, digits=4)
# #matrix = confusion_matrix(p,q)
# print(report)


In [ ]:
# new_df_test['predicted'] = q
# new_df_test['predicted'].replace({0:'question',1:'answer',2:'header',3:'other'})
# new_df_test.to_pickle('/content/funsd_results.pkl')

#Bert Only
**Notice: you can only use the provided code in this project only. You can not directly copy this code to other project or assignment.**

## Environment Setup
Import key libraries and working envorinments. 

In [ ]:
!pip install transformers==4.2.2

In [ ]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, RobertaModel
#from transformers.configuration_bert import BertConfig
import logging
logging.basicConfig(level=logging.ERROR)

In [ ]:
import gc
gc.collect()
with torch.no_grad():
    torch.cuda.empty_cache()
!nvidia-smi

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd

# Authenticate
drive = None
def authenticate():
  global drive
  
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

#Download files
def downloadFiles(fileIds):
  authenticate()
  
  for fileId in fileIds:    
    
    downloaded = drive.CreateFile({"id": fileId[1]})
    downloaded.GetContentFile(fileId[0])

##Generate Training and Validation Dataset (Only need to run if you don't have it)

In [ ]:
# try:
#   _ = open("DL_dataset.csv", "r")
# except:
#     downloadFiles([["DL_dataset.csv", "1-8WAgkLmCpk6_q1-hjgSapyrIKZi8xsW"]])



In [ ]:
# df_data = pd.read_csv('DL_dataset.csv')
# df_data["label"].value_counts()

In [ ]:
# label_transform = {"label": {"title":0, "section":1, "table":2, "text":3}}
# df_data = df_data.replace(label_transform)
# df_data["label"].value_counts()

In [ ]:
# train_df, test_df = np.split(df_data.sample(frac=1, random_state=42), [int(.7*len(df_data))])

# new_df = train_df[['text', 'label','visual_feature']]
# new_df_test = test_df[['text', 'label','visual_feature']]
# print(len(new_df))
# print(len(new_df_test))

In [ ]:
# #save
# new_df.to_csv("/content/drive/MyDrive/CS62-1_DFU/week 12/train_data.csv",index=False)


In [ ]:
# new_df_test.to_csv("/content/drive/MyDrive/cap2/test_data.csv", index=False)

In [ ]:
# new_df.to_csv("/content/train_data.csv",index=False)
# new_df_test.to_csv("/content/test_data.csv", index=False)

##Get Training and Validation Dataset

In [ ]:
# Setting up the device for GPU usage
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
try:
  _ = open("test_data.csv", "r")
except:
  downloadFiles([["test_data.csv", "1-0zzjHHuvraIpp4GMtiNAaLOB8t68dej"]])

try:
  _ = open("train_data.csv", "r")
except:
  downloadFiles([["train_data.csv", "1XYPMwAtLobx4wFa1VB5K7S2O7MxWxtk1"]])

new_df = pd.read_csv('train_data.csv')

new_df_test = pd.read_csv('test_data.csv')

## Data Preprocessing

In [ ]:
from transformers import AutoTokenizer,AutoModel
import torch


tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# Defining some key variables that will be used later on in the training
MAX_LEN = 512
TRAIN_BATCH_SIZE = 12
VALID_BATCH_SIZE = TRAIN_BATCH_SIZE*2
# EPOCHS = 1
LEARNING_RATE = 2e-05
# Change the pre-trained bert model
#tokenizer = BertTokenizer.from_pretrained('roberta-base') #Cased 

In [ ]:
class SentimentData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = self.data.text
        self.targets = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float),
        }

In [ ]:
train_size = 1
train_data=new_df.sample(frac=train_size,random_state=200)
train_data = train_data.reset_index(drop=True)


print("FULL Dataset: {}".format(new_df.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(new_df_test.shape))

training_set = SentimentData(train_data, tokenizer, MAX_LEN)
test_set = SentimentData(new_df_test,tokenizer,MAX_LEN)

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
vali_loader = DataLoader(test_set, **test_params)

## Define the proposed classifiers

In [ ]:
class BertClass(torch.nn.Module):
    def __init__(self):
        super(BertClass, self).__init__()
        #BERT/BERT Large/Roberta/LayoutLM/Distill BERT/ALBERT

        self.l1 = AutoModel.from_pretrained('bert-base-uncased')# BERT large

        self.pre_classifier = torch.nn.Linear(768, 768)


        self.classifier = torch.nn.Linear(768, 4)


    def forward(self,input_ids,attention_mask,token_type_ids):
        output_1 = self.l1(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        hidden_state = output_1[0]
        pooler = hidden_state[:, 0] 

        output = self.classifier(pooler)
        return output

In [ ]:
model = BertClass()
model.to(device)

## Training 

In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=1e-05) # change learning rate

def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the distilbert model
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    output = []
    model.train()
    count = 0
    for _,data in tqdm(enumerate(training_loader, 0)):
        count += 1
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        outputs = model(ids, mask, token_type_ids)

        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(outputs.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%5000==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 

        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()
        if isinstance(BATCH_NUM,int) and count > BATCH_NUM:
          break
    
    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch, epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch, epoch_accu}")
    torch.save(model.state_dict(), '/content/model.pt')
    
    return

## Testing

In [ ]:
class SentimentData_test(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
        }

### load the test datasets

In [ ]:
new_df_true_test = new_df_test[['text']]

In [ ]:
test = SentimentData_test(new_df_true_test,tokenizer, MAX_LEN)
testing_loader = DataLoader(test, **test_params)

In [ ]:
def test_label_generator(model, testing_loader):
    model.eval()
    n_correct = 0; n_wrong = 0; total = 0; tr_loss=0; nb_tr_steps=0; nb_tr_examples=0
    output_list = []
    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype=torch.long)

            outputs = model(ids, mask, token_type_ids)
            
            big_val, big_idx = torch.max(outputs.data, dim=1)
            output_list = output_list + list(big_idx)

            nb_tr_steps += 1
            
    return output_list


## Train & Test

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

EPOCHS = 15
BATCH_NUM = int(len(training_loader)/2)

for epoch in range(EPOCHS):
  output = train(epoch)
  test_output = test_label_generator(model, testing_loader)
  # Get the predicted category id for selected test dataset.
  q = []
  for p in test_output:
    q.append(p.cpu().numpy().tolist())
  p = new_df_test['label'].tolist()
  report = classification_report(p,q, digits=4)
  #matrix = confusion_matrix(p,q)
  print(report)


In [ ]:
#separate test
# test_output = test_label_generator(model, testing_loader)
# # Get the predicted category id for selected test dataset.
# q = []
# for p in test_output:
#   q.append(p.cpu().numpy().tolist())
# p = new_df_test['label'].tolist()
# report = classification_report(p,q, digits=4)
# #matrix = confusion_matrix(p,q)
# print(report)

In [ ]:
# output = test_label_generator(model, testing_loader)

In [ ]:
# Get the predicted category id for selected test dataset.
# q = []
# for p in output:
#   q.append(p.cpu().numpy().tolist())
# print(q)

In [ ]:
# p = new_df_test['label'].tolist()

In [ ]:
# from sklearn.metrics import classification_report, confusion_matrix
# report = classification_report(p,q, digits=4)
# #matrix = confusion_matrix(p,q)
# print(report)


In [ ]:
# new_df_test['predicted'] = q
# new_df_test['predicted'].replace({0:'question',1:'answer',2:'header',3:'other'})
# new_df_test.to_pickle('/content/funsd_results.pkl')

# Visualize Your Results

In [ ]:
! wget https://guillaumejaume.github.io/FUNSD/dataset.zip
! unzip dataset.zip && mv dataset data && rm -rf dataset.zip __MACOSX

In [ ]:
import pandas as pd
import json
import pickle
import os
from PIL import Image
# Show the image in ipynb
from IPython.display import clear_output, Image, display
import PIL.Image
def showarray(a, fmt='jpg'):
    a = np.uint8(np.clip(a, 0, 255))
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))

In [ ]:
with open('scene_graph.json') as f:
  img_info = json.load(f)

In [ ]:
our_df = pd.read_pickle('/content/funsd_results.pkl')

In [ ]:
our_df.head()

In [ ]:
label_list= our_df['label'].tolist()
target_list = our_df['predicted'].tolist()

In [ ]:
check_list_name = []
check_list_id = []
i = 0
for img in img_info:
  j = 0
  for obj in img_info[img]['objects']:
    objt = img_info[img]['objects'][obj]
    objt['target'] = label_list[i]
    objt['predict'] = target_list[i]

    if objt['target'] != objt['predict']:
      j += 1
      if (img_info[img]['file_name'] not in check_list_name):
        check_list_name.append(img_info[img]['file_name'])
        check_list_id.append(img)
    i+=1

In [ ]:
m =4

In [ ]:
from PIL import Image, ImageDraw, ImageFont
path = '/content/data/testing_data/images/'+ check_list_name[m]+'.png'
image = Image.open(path)
image = image.convert("RGB")

In [ ]:
draw = ImageDraw.Draw(image, "RGBA")

font = ImageFont.load_default()

label2color = {0:(0,255,0,80), 1:(255,0,0,80), 2:(255,255,0,80), 3:(15,15,200,80)}
for obj in img_info[check_list_id[m]]['objects']:
  annotation = img_info[check_list_id[m]]['objects'][obj]
  label = annotation['target']
  x = annotation['box'][0]
  y = annotation['box'][1]
  w =  annotation['box'][2]
  h =  annotation['box'][3]
  general_box = [x,y,w,h]
  draw.rectangle(general_box, fill = label2color[label])
  #draw.text((annotation['box'][0] , annotation['box'][1]-10), label, fill=label2color[label], font=font)

image

In [ ]:
from PIL import Image, ImageDraw, ImageFont
path = '/content/data/testing_data/images/'+ check_list_name[m]+'.png'
image = Image.open(path)
image = image.convert("RGB")

In [ ]:
draw = ImageDraw.Draw(image, "RGBA")

font = ImageFont.load_default()

label2color = {0:(0,255,0,80), 1:(255,0,0,80), 2:(255,255,0,80), 3:(15,15,200,80)}

for obj in img_info[check_list_id[m]]['objects']:
  annotation = img_info[check_list_id[m]]['objects'][obj]
  label = annotation['predict']
  x = annotation['box'][0]
  y = annotation['box'][1]
  w =  annotation['box'][2]
  h =  annotation['box'][3]
  general_box = [x,y,w,h]
  draw.rectangle(general_box, fill = label2color[label])

image